### A movie recommender system 
### Create a content-based recommender system to suggeat movies to viewers based on their previous history

In [29]:
#Import libraries
import numpy as np
import pandas as pd

#Load dataset
df = pd.read_csv('tmdb_movies_data.csv')
# df.head()

In [32]:
movies = df[['id','cast','director','genres','overview','original_title','keywords']]    

#Drop null values
movies.isnull().sum()
movies.dropna(inplace = True)

/var/folders/08/p0v319w963s3b56bqpb1t1tr0000gq/T/ipykernel_11058/1095340313.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace = True)


In [35]:
#Transform data to standard format
def convert(obj):
    s = list(obj)
    for i in range(len(s)):
        if s[i] == '|':
         s[i] = " "
    
    temp_str = "".join(s)
    temp_list = temp_str.split()

    return temp_list

# Apply the convert function to all the columns
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['cast'] = movies['cast'].apply(convert)
movies['director'] = movies['director'].apply(convert)

# Stem the data to reduce words to their base form
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))

    return " ".join(y)

/var/folders/08/p0v319w963s3b56bqpb1t1tr0000gq/T/ipykernel_11058/249965718.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['genres'] = movies['genres'].apply(convert)
/var/folders/08/p0v319w963s3b56bqpb1t1tr0000gq/T/ipykernel_11058/249965718.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['keywords'] = movies['keywords'].apply(convert)
/var/folders/08/p0v319w963s3b56bqpb1t1tr0000gq/T/ipykernel_11058/249965718.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [36]:
#Create the recommender model
# Create a new column that combines all charecteristics of the movie
movies['tags'] = movies['cast'] + movies['genres'] + movies['director'] + movies['overview'] + movies['keywords']

# Create a new dataframe consisting of id, original title, and tags
new_df = movies[['id','original_title','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

# Apply the stem function to the tags column of your dataframe
new_df['tags'] = new_df['tags'].apply(stem)

/var/folders/08/p0v319w963s3b56bqpb1t1tr0000gq/T/ipykernel_11058/3687707715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['cast'] + movies['genres'] + movies['director'] + movies['overview'] + movies['keywords']
/var/folders/08/p0v319w963s3b56bqpb1t1tr0000gq/T/ipykernel_11058/3687707715.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
/var/folders/08/p0v319w963s3b56bqpb1t1tr0000gq/T/ipykernel_11058/3687707715.py:10:

In [ ]:
# Perform text vectorization after disregarding stop words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words='english')

# Transform the SciPy sparse matrix to NumPy array form
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
# Calculate the cosine similarity between the vectors
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

In [ ]:
# Take the pickle dump of the results for later use
import pickle

pickle.dump(new_df,open('movies.pkl','wb'))
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))